In [27]:
import keras
import os
import librosa
import numpy as np
from keras.models import load_model
import tqdm
from numba import jit

In [2]:
model=load_model('model_60_UTC8_20191209_16_49_48')











Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 118, 32)           160       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 115, 64)           8256      
_________________________________________________________________
flatten_1 (Flatten)          (None, 7360)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 121)               890681    
Total params: 899,097
Trainable params: 899,097
Non-trainable params: 0
_________________________________________________________________


In [16]:
def load_audio(fdir, window_sz=20, batchsz=500):
    xb=[]
    for f in os.listdir(fdir):
        musica, sr = librosa.load(fdir+'/'+f, sr=44100)
        print('LOAD', f)
        #[win-1-win] is our training window
        
        for i in range(0, musica.shape[0]-1-2*window_sz):
            xb.append(musica[i:i+1+2*window_sz].reshape(-1,1))
            if len(xb) == batchsz:
                yield np.array(xb)
                xb=[]
#         if len(xb)>0:
#             yield np.array(xb, dtype=np.float16)

In [13]:
wsz=60
bsz=10000

In [14]:
batches=[i for i in load_audio('real_case', window_sz=60, batchsz=10000)]

LOAD 9-21-furtwangler-957.wav
LOAD 9-27-furtwangler-1052.wav
LOAD 9-49-furtwangler-1074.wav


In [17]:
len(batches)

396

In [18]:
fixed=[]
for i in tqdm.tqdm(batches):
    y=model.predict(i)
    fixed.append(y)

100%|█████████████| 396/396 [02:01<00:00,  3.26it/s]


In [19]:
batches[0].shape

(10000, 121, 1)

In [29]:
fixed_musica=[]
for idx, i in enumerate(fixed):
    s=0.9*(batches[idx])+0.1*(fixed[idx])
    fixed_musica.append(np.pad(s.ravel(), ((0, bsz-i.shape[0]),)))

ValueError: operands could not be broadcast together with shapes (10000,121,1) (10000,1) 

In [25]:
yarray=np.array(fixed).reshape(-1,2*wsz+1)

In [26]:
yarray.shape

(3960000, 121)

In [28]:
fixed=np.zeros(yarray.shape[0]+2*wsz+1)

In [30]:
for idx, i in enumerate(yarray):
    fixed[idx:idx+yarray.shape[1]]+=i

In [32]:
fixed/=float(2*wsz)

In [33]:
np.max(fixed)

0.6487833975760404

In [35]:
np.max(batches[100])

0.1279602

In [36]:
librosa.output.write_wav('desmooth.wav', fixed, 44100)